In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Load MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Resize and convert to 3 channels
def preprocess(images):
    images = np.expand_dims(images, axis=-1)
    images = tf.image.resize(images, (224,224))
    images = tf.image.grayscale_to_rgb(images)
    images = tf.keras.applications.vgg16.preprocess_input(images)
    return images

x_train = preprocess(x_train[:2000])
y_train = y_train[:2000]

# Load pretrained VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
x = Flatten()(base_model.output)
model = Model(inputs=base_model.input, outputs=x)

# Extract features BEFORE transfer learning
features_before = model.predict(x_train, batch_size=32)

# Add classifier for transfer learning
x = Dense(256, activation='relu')(model.output)
output = Dense(10, activation='softmax')(x)
transfer_model = Model(inputs=model.input, outputs=output)

transfer_model.compile(optimizer='adam',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Train on MNIST
transfer_model.fit(x_train, y_train, epochs=3, batch_size=32)

# Extract features AFTER transfer learning
feature_model_after = Model(inputs=transfer_model.input,
                            outputs=transfer_model.layers[-3].output)

features_after = feature_model_after.predict(x_train)

# ------------------------
# PCA Visualization
# ------------------------
pca = PCA(n_components=2)

pca_before = pca.fit_transform(features_before)
pca_after = pca.fit_transform(features_after)

plt.figure()
plt.scatter(pca_before[:,0], pca_before[:,1], c=y_train)
plt.title("PCA Before Transfer Learning")
plt.show()

plt.figure()
plt.scatter(pca_after[:,0], pca_after[:,1], c=y_train)
plt.title("PCA After Transfer Learning")
plt.show()

# ------------------------
# t-SNE Visualization
# ------------------------
tsne = TSNE(n_components=2, perplexity=30)

tsne_before = tsne.fit_transform(features_before)
tsne_after = tsne.fit_transform(features_after)

plt.figure()
plt.scatter(tsne_before[:,0], tsne_before[:,1], c=y_train)
plt.title("t-SNE Before Transfer Learning")
plt.show()

plt.figure()
plt.scatter(tsne_after[:,0], tsne_after[:,1], c=y_train)
plt.title("t-SNE After Transfer Learning")
plt.show()